In [1]:
!pip install auto-ts
!pip install prophet
!pip install pmdarima
!pip install xgboost
!pip install pandas_market_calendars
!pip install lightgbm==3.3.5


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.3/962.3 KB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.1/630.1 KB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.3/322.3 KB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import yfinance as yf
import time
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import auto_ts as at
from math import floor,ceil
import pmdarima as pm
import statsmodels.api as sm
from prophet import Prophet
from prophet.diagnostics import cross_validation
from sklearn.model_selection import ParameterGrid
import auto_ts as at
import itertools
import pandas_market_calendars as mcal
import numpy as np
import matplotlib.pyplot as plt 
from datetime import datetime, timedelta


Imported auto_timeseries version:0.0.69. Call by using:
model = auto_timeseries(score_type='rmse',
        time_interval='M', non_seasonal_pdq=None, seasonality=False,
        seasonal_period=12, model_type=['best'], verbose=2, dask_xgboost_flag=0)
model.fit(traindata, ts_column,target)
model.predict(testdata, model='best')



In [3]:
scope = ['https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive']

creds = Credentials.from_service_account_file("/work/cuenta de servicio.json", scopes=scope)
client = gspread.authorize(creds)

In [4]:
worksheet_data = client.open("SP_500_data_clean").get_worksheet(0).get_all_records()
worksheet = client.open("SP_500_data_clean").get_worksheet(0)

In [25]:
data = get_as_dataframe(worksheet)
data.set_index("Date", inplace=True)
data.index = pd.to_datetime(data.index)
#data.reset_index(inplace=True)

In [15]:
data

,Date,AAPL,MSFT,AMZN,TSLA,META,UNH,V,NVDA,JNJ,...,OGN,XRAY,WYNN,RL,DXC,ALK,PENN,VNO,PVH,NCLH
0,2022-01-03,180.959732,331.642456,170.404495,399.926666,338.540009,497.457550,219.767578,301.005585,168.227264,...,30.040031,56.309315,87.769997,118.235039,33.230000,54.070000,50.639999,40.806446,107.892570,22.180000
1,2022-01-04,178.663086,325.955780,167.522003,383.196655,336.529999,486.186798,220.789841,292.701202,167.776154,...,30.387093,56.596359,87.489998,121.412865,34.110001,55.450001,49.770000,42.232529,111.336586,22.469999
2,2022-01-05,173.910660,313.443024,164.356995,362.706665,324.170013,484.988434,218.348312,275.852661,168.894150,...,29.982187,55.685745,84.250000,119.418159,33.470001,54.770000,46.610001,41.449585,109.270172,21.660000
3,2022-01-06,171.007507,310.966187,163.253998,354.899994,332.459991,465.140808,218.100189,281.588776,168.315536,...,30.377451,55.646156,84.639999,117.384354,33.730000,54.459999,46.860001,42.018154,109.939011,21.340000
4,2022-01-07,171.176544,311.124695,162.554001,342.320007,331.790009,454.196930,215.331146,272.285095,170.590744,...,31.245102,55.824318,84.550003,115.438538,33.340000,55.860001,47.279999,42.661285,106.774513,22.200001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,2023-04-21,165.020004,285.760010,106.959999,165.080002,212.889999,483.820007,234.050003,271.190002,162.690002,...,24.110001,41.090000,114.110001,115.110001,24.129999,44.410000,29.639999,15.240000,87.000000,13.040000
327,2023-04-24,165.330002,281.769989,106.209999,162.550003,212.789993,488.940002,232.759995,270.420013,163.679993,...,24.030001,41.439999,114.480003,116.129997,24.160000,44.049999,29.590000,15.330000,87.889999,13.050000
328,2023-04-25,163.770004,275.420013,102.570000,160.669998,207.550003,491.920013,229.589996,262.410004,165.179993,...,23.590000,40.970001,111.629997,112.430000,23.290001,42.500000,28.540001,14.910000,83.320000,12.730000
329,2023-04-26,163.759995,295.369995,104.980003,153.750000,209.399994,482.950012,228.149994,269.559998,162.619995,...,23.700001,40.930000,109.449997,111.989998,22.860001,42.240002,28.129999,14.930000,83.120003,12.460000


In [26]:
# dividir en set de train tes con 80% de train

largo_train = floor(len(data)*0.8)

data_train = data[:largo_train]
data_test = data[largo_train:]

largo_train

264

In [8]:

# crear un calendario con los días en los que la bolsa de Nueva York está abierta

nyse = mcal.get_calendar('NYSE')
schedule = nyse.schedule(start_date='2022-01-01', end_date='2022-12-31')

# crear un DataFrame con todas las fechas entre start_date y end_date, incluyendo sábados y domingos
idx = pd.date_range(start='2022-01-01', end=f'{data.tail(1).index.year[0]}-{data.tail(1).index.month[0]}-{data.tail(1).index.day[0]}', freq='D')
idx = idx[~idx.isin(schedule.index)]


In [9]:
nl = pd.DataFrame(idx,columns=["ds"])
nl["holiday"] = "no_labour_day"
nl = nl[["holiday","ds"]]



In [10]:
f_date = data.tail(1).index + timedelta(60)
f_year = f_date.year[0]
f_month = f_date.month[0]
f_day = f_date.day[0]




0      170.404495
1      167.522003
2      164.356995
3      163.253998
4      162.554001
          ...    
259     98.120003
260     96.050003
261     95.459999
262     93.680000
263     97.250000
Name: y, Length: 264, dtype: float64

In [18]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from itertools import product
from tqdm import tqdm

def arima_model_selection(price, date, seasonal_periods=7):
    # Convertimos los datos a un DataFrame con las fechas como índice
    data = pd.DataFrame({"date": date, "price": price})
    data = data.set_index(pd.to_datetime(data["date"])).drop("date", axis=1)
    
    # Seleccionamos los hiperparámetros a evaluar
    p = range(0, 3) # Orden del proceso autoregresivo
    d = range(0, 2) # Orden de diferenciación
    q = range(0, 3) # Orden del proceso de media móvil
    params = list(product(p, d, q))
    
    # Realizamos una búsqueda exhaustiva para encontrar los mejores hiperparámetros
    best_aic = np.inf
    for param in tqdm(params):
        try:
            model = ARIMA(data, order=param).fit()
            aic = model.aic
            if aic < best_aic:
                best_aic = aic
                best_param = param
                best_model = model
        except:
            continue
    
    # Imprimimos los resultados
    print("Mejor modelo ARIMA encontrado:")
    print("Parámetros (p, d, q):", best_param)
    print("AIC:", best_aic)
    
    # Ajustamos el modelo con los mejores hiperparámetros
    return best_model

In [44]:
model = sm.tsa.ARIMA(data_train["AMZN"], order=(0,1,0))

In [45]:
arima_result = model.fit()

In [46]:
start_date = '2022-01-03'
end_date = '2023-04-27'
pred_range = pd.date_range(start=start_date, end=end_date, freq='D')

In [50]:
predictions = arima_result.predict(start=data_test.index, end=pred_range[-10], dynamic=False)

KeyError: 'The `end` argument could not be matched to a location related to the index of the data.'

In [48]:
pred_range[-1]

Timestamp('2023-04-27 00:00:00')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1904e425-e017-401a-bf5a-7f8e011fcf07' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>